In [1]:
#Indicative snapsot of how the baselines' experiments were performed with the use of darts library. In the current notebook,
#the experiments with NBEATS model are presented. The same procedure was followed for the Exponential Smoothing and the 
#Theta Method,

In [ ]:
import warnings
import torch
warnings.simplefilter('ignore')

from torch.nn.modules.loss import MSELoss
import pandas as pd
from pandas import Series
import numpy as np
from numpy import hstack
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,3)
from datetime import datetime, date 
from numpy import array
import matplotlib.pyplot as plt

import missingno
from typing import Iterable, List
import joblib
import darts
from darts.metrics import mape, r2_score, mse, mae, rmse
from darts import TimeSeries
from tqdm import tqdm
from darts.models import NBEATSModel
!jupyter nbextension enable --py widgetsnbextension

In [4]:
#DATA PREPARATION

In [5]:
#COUNTRIES DATASET
countries = pd.read_csv("energy_data.csv")
countries

,utc_timestamp,cet_cest_timestamp,AT_load_actual_entsoe_transparency,AT_load_forecast_entsoe_transparency,AT_price_day_ahead,AT_solar_generation_actual,AT_wind_onshore_generation_actual,BE_load_actual_entsoe_transparency,BE_load_forecast_entsoe_transparency,BE_solar_generation_actual,...,SI_load_actual_entsoe_transparency,SI_load_forecast_entsoe_transparency,SI_solar_generation_actual,SI_wind_onshore_generation_actual,SK_load_actual_entsoe_transparency,SK_load_forecast_entsoe_transparency,SK_solar_generation_actual,SK_wind_onshore_generation_actual,UA_load_actual_entsoe_transparency,UA_load_forecast_entsoe_transparency
0,2014-12-31T23:00:00Z,2015-01-01T00:00:00+0100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-01T00:00:00Z,2015-01-01T01:00:00+0100,5946.0,6701.0,35.0,NaN,69.0,9484.0,9897.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-01-01T01:00:00Z,2015-01-01T02:00:00+0100,5726.0,6593.0,45.0,NaN,64.0,9152.0,9521.0,NaN,...,1045.47,816.0,NaN,1.17,2728.0,2860.0,3.8,NaN,NaN,NaN
3,2015-01-01T02:00:00Z,2015-01-01T03:00:00+0100,5347.0,6482.0,41.0,NaN,65.0,8799.0,9135.0,NaN,...,1004.79,805.0,NaN,1.04,2626.0,2810.0,3.8,NaN,NaN,NaN
4,2015-01-01T03:00:00Z,2015-01-01T04:00:00+0100,5249.0,6454.0,38.0,NaN,64.0,8567.0,8909.0,NaN,...,983.79,803.0,NaN,1.61,2618.0,2780.0,3.8,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50396,2020-09-30T19:00:00Z,2020-09-30T21:00:00+0200,6661.0,6656.0,NaN,NaN,1847.0,9402.0,9720.0,NaN,...,1506.95,1530.0,5.32,0.44,3324.0,3170.0,1.7,1.1,NaN,NaN
50397,2020-09-30T20:00:00Z,2020-09-30T22:00:00+0200,6336.0,6310.0,NaN,NaN,1723.0,9219.0,9418.0,NaN,...,1391.15,1388.0,5.57,0.62,3099.0,3010.0,1.7,1.0,NaN,NaN
50398,2020-09-30T21:00:00Z,2020-09-30T23:00:00+0200,5932.0,5813.0,NaN,NaN,1771.0,8763.0,8873.0,NaN,...,1250.17,1274.0,5.58,0.66,2940.0,2890.0,1.6,1.0,NaN,NaN
50399,2020-09-30T22:00:00Z,2020-10-01T00:00:00+0200,5628.0,5424.0,NaN,NaN,1779.0,8131.0,8285.0,NaN,...,1206.82,1178.0,5.61,0.21,2823.0,2740.0,1.7,1.2,NaN,NaN


In [6]:
countries['utc_timestamp']= pd.to_datetime(countries['utc_timestamp']).dt.tz_localize(None)

countries['year'] = countries['utc_timestamp'].dt.year
countries['month'] = countries['utc_timestamp'].dt.month
countries['week'] = countries['utc_timestamp'].dt.week
countries['day'] = countries['utc_timestamp'].dt.day

In [7]:
values = countries['month'].index
sine = np.sin(values*2*np.pi/len(values))
cosine = np.cos(values*2*np.pi/len(values))
countries['month_sin'] = sine
countries['month_cos'] = cosine

countries

,utc_timestamp,cet_cest_timestamp,AT_load_actual_entsoe_transparency,AT_load_forecast_entsoe_transparency,AT_price_day_ahead,AT_solar_generation_actual,AT_wind_onshore_generation_actual,BE_load_actual_entsoe_transparency,BE_load_forecast_entsoe_transparency,BE_solar_generation_actual,...,SK_solar_generation_actual,SK_wind_onshore_generation_actual,UA_load_actual_entsoe_transparency,UA_load_forecast_entsoe_transparency,year,month,week,day,month_sin,month_cos
0,2014-12-31 23:00:00,2015-01-01T00:00:00+0100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2014,12,1,31,0.000000,1.0
1,2015-01-01 00:00:00,2015-01-01T01:00:00+0100,5946.0,6701.0,35.0,NaN,69.0,9484.0,9897.0,NaN,...,NaN,NaN,NaN,NaN,2015,1,1,1,0.000125,1.0
2,2015-01-01 01:00:00,2015-01-01T02:00:00+0100,5726.0,6593.0,45.0,NaN,64.0,9152.0,9521.0,NaN,...,3.8,NaN,NaN,NaN,2015,1,1,1,0.000249,1.0
3,2015-01-01 02:00:00,2015-01-01T03:00:00+0100,5347.0,6482.0,41.0,NaN,65.0,8799.0,9135.0,NaN,...,3.8,NaN,NaN,NaN,2015,1,1,1,0.000374,1.0
4,2015-01-01 03:00:00,2015-01-01T04:00:00+0100,5249.0,6454.0,38.0,NaN,64.0,8567.0,8909.0,NaN,...,3.8,NaN,NaN,NaN,2015,1,1,1,0.000499,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50396,2020-09-30 19:00:00,2020-09-30T21:00:00+0200,6661.0,6656.0,NaN,NaN,1847.0,9402.0,9720.0,NaN,...,1.7,1.1,NaN,NaN,2020,9,40,30,-0.000623,1.0
50397,2020-09-30 20:00:00,2020-09-30T22:00:00+0200,6336.0,6310.0,NaN,NaN,1723.0,9219.0,9418.0,NaN,...,1.7,1.0,NaN,NaN,2020,9,40,30,-0.000499,1.0
50398,2020-09-30 21:00:00,2020-09-30T23:00:00+0200,5932.0,5813.0,NaN,NaN,1771.0,8763.0,8873.0,NaN,...,1.6,1.0,NaN,NaN,2020,9,40,30,-0.000374,1.0
50399,2020-09-30 22:00:00,2020-10-01T00:00:00+0200,5628.0,5424.0,NaN,NaN,1779.0,8131.0,8285.0,NaN,...,1.7,1.2,NaN,NaN,2020,9,40,30,-0.000249,1.0


In [8]:
countries.index = pd.to_datetime(countries['utc_timestamp']).dt.tz_localize(None)
mask1 = (countries['utc_timestamp'] > "2015-01-01 00:00:00") & (countries['utc_timestamp'] <= "2019-12-31 19:00:00")
countries = countries.loc[mask1]
countries

,utc_timestamp,cet_cest_timestamp,AT_load_actual_entsoe_transparency,AT_load_forecast_entsoe_transparency,AT_price_day_ahead,AT_solar_generation_actual,AT_wind_onshore_generation_actual,BE_load_actual_entsoe_transparency,BE_load_forecast_entsoe_transparency,BE_solar_generation_actual,...,SK_solar_generation_actual,SK_wind_onshore_generation_actual,UA_load_actual_entsoe_transparency,UA_load_forecast_entsoe_transparency,year,month,week,day,month_sin,month_cos
utc_timestamp,,,,,,,,,,,,,,,,,,,,,
2015-01-01 01:00:00,2015-01-01 01:00:00,2015-01-01T02:00:00+0100,5726.0,6593.0,45.0,NaN,64.0,9152.0,9521.0,NaN,...,3.8,NaN,NaN,NaN,2015,1,1,1,0.000249,1.000000
2015-01-01 02:00:00,2015-01-01 02:00:00,2015-01-01T03:00:00+0100,5347.0,6482.0,41.0,NaN,65.0,8799.0,9135.0,NaN,...,3.8,NaN,NaN,NaN,2015,1,1,1,0.000374,1.000000
2015-01-01 03:00:00,2015-01-01 03:00:00,2015-01-01T04:00:00+0100,5249.0,6454.0,38.0,NaN,64.0,8567.0,8909.0,NaN,...,3.8,NaN,NaN,NaN,2015,1,1,1,0.000499,1.000000
2015-01-01 04:00:00,2015-01-01 04:00:00,2015-01-01T05:00:00+0100,5309.0,6609.0,35.0,NaN,64.0,8487.0,8806.0,NaN,...,3.8,NaN,NaN,NaN,2015,1,1,1,0.000623,1.000000
2015-01-01 05:00:00,2015-01-01 05:00:00,2015-01-01T06:00:00+0100,5574.0,6543.0,35.0,NaN,84.0,8428.0,8805.0,NaN,...,3.8,NaN,NaN,NaN,2015,1,1,1,0.000748,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-31 15:00:00,2019-12-31 15:00:00,2019-12-31T16:00:00+0100,7528.0,8009.0,NaN,19.0,2243.0,10081.0,10308.0,13.14,...,8.8,0.9,20672.0,21020.0,2019,12,1,31,-0.731768,0.681554
2019-12-31 16:00:00,2019-12-31 16:00:00,2019-12-31T17:00:00+0100,7807.0,8248.0,NaN,7.0,2048.0,10381.0,10870.0,0.00,...,0.6,0.9,20357.0,21160.0,2019,12,1,31,-0.731683,0.681645
2019-12-31 17:00:00,2019-12-31 17:00:00,2019-12-31T18:00:00+0100,7721.0,8090.0,NaN,4.0,2088.0,10053.0,10842.0,0.00,...,0.5,1.2,19816.0,20960.0,2019,12,1,31,-0.731598,0.681737


In [9]:
countries = countries [['utc_timestamp','AT_solar_generation_actual','BE_solar_generation_actual','BG_solar_generation_actual', 'CH_solar_generation_actual','CZ_solar_generation_actual','DE_solar_generation_actual','DK_solar_generation_actual','ES_solar_generation_actual','FR_solar_generation_actual','GR_solar_generation_actual','HR_solar_generation_actual','HU_solar_generation_actual','IT_solar_generation_actual','LT_solar_generation_actual', 'NL_solar_generation_actual','PL_solar_generation_actual','PT_solar_generation_actual','RO_solar_generation_actual','SI_solar_generation_actual','SK_solar_generation_actual', 'GB_UKM_solar_generation_actual', 'year', 'month', 'week', 'day', 'month_sin', 'month_cos']]
countries

,utc_timestamp,AT_solar_generation_actual,BE_solar_generation_actual,BG_solar_generation_actual,CH_solar_generation_actual,CZ_solar_generation_actual,DE_solar_generation_actual,DK_solar_generation_actual,ES_solar_generation_actual,FR_solar_generation_actual,...,RO_solar_generation_actual,SI_solar_generation_actual,SK_solar_generation_actual,GB_UKM_solar_generation_actual,year,month,week,day,month_sin,month_cos
utc_timestamp,,,,,,,,,,,,,,,,,,,,,
2015-01-01 01:00:00,2015-01-01 01:00:00,NaN,NaN,NaN,0.05,NaN,NaN,NaN,50.0,NaN,...,NaN,NaN,3.8,NaN,2015,1,1,1,0.000249,1.000000
2015-01-01 02:00:00,2015-01-01 02:00:00,NaN,NaN,NaN,0.07,NaN,NaN,NaN,50.0,NaN,...,NaN,NaN,3.8,NaN,2015,1,1,1,0.000374,1.000000
2015-01-01 03:00:00,2015-01-01 03:00:00,NaN,NaN,NaN,0.07,NaN,NaN,NaN,42.0,NaN,...,NaN,NaN,3.8,NaN,2015,1,1,1,0.000499,1.000000
2015-01-01 04:00:00,2015-01-01 04:00:00,NaN,NaN,NaN,0.07,NaN,NaN,0.01,34.0,NaN,...,NaN,NaN,3.8,NaN,2015,1,1,1,0.000623,1.000000
2015-01-01 05:00:00,2015-01-01 05:00:00,NaN,NaN,NaN,0.07,NaN,NaN,0.01,34.0,NaN,...,NaN,NaN,3.8,NaN,2015,1,1,1,0.000748,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-31 15:00:00,2019-12-31 15:00:00,19.0,13.14,0.0,2.40,9.61,46.0,0.73,2999.0,454.0,...,0.0,0.35,8.8,99.0,2019,12,1,31,-0.731768,0.681554
2019-12-31 16:00:00,2019-12-31 16:00:00,7.0,0.00,0.0,0.00,0.00,0.0,0.00,1250.0,109.0,...,0.0,0.05,0.6,0.0,2019,12,1,31,-0.731683,0.681645
2019-12-31 17:00:00,2019-12-31 17:00:00,4.0,0.00,0.0,0.00,0.00,0.0,0.00,182.0,0.0,...,0.0,0.05,0.5,0.0,2019,12,1,31,-0.731598,0.681737


In [10]:
#WEATHER DATASET
weather = pd.read_csv("weather_data.csv")
weather.index = pd.to_datetime(weather['utc_timestamp']).dt.tz_localize(None)
weather

,utc_timestamp,AT_temperature,AT_radiation_direct_horizontal,AT_radiation_diffuse_horizontal,BE_temperature,BE_radiation_direct_horizontal,BE_radiation_diffuse_horizontal,BG_temperature,BG_radiation_direct_horizontal,BG_radiation_diffuse_horizontal,...,RO_radiation_diffuse_horizontal,SE_temperature,SE_radiation_direct_horizontal,SE_radiation_diffuse_horizontal,SI_temperature,SI_radiation_direct_horizontal,SI_radiation_diffuse_horizontal,SK_temperature,SK_radiation_direct_horizontal,SK_radiation_diffuse_horizontal
utc_timestamp,,,,,,,,,,,,,,,,,,,,,
1980-01-01 00:00:00,1980-01-01T00:00:00Z,-3.640,0.0,0.0,-0.720,0.0,0.0,4.664,0.0,0.0,...,0.0,-3.945,0.0,0.0,-3.055,0.0,0.0,-4.648,0.0,0.0
1980-01-01 01:00:00,1980-01-01T01:00:00Z,-3.803,0.0,0.0,-1.165,0.0,0.0,4.052,0.0,0.0,...,0.0,-4.053,0.0,0.0,-3.272,0.0,0.0,-4.554,0.0,0.0
1980-01-01 02:00:00,1980-01-01T02:00:00Z,-3.969,0.0,0.0,-1.434,0.0,0.0,3.581,0.0,0.0,...,0.0,-4.129,0.0,0.0,-3.639,0.0,0.0,-4.455,0.0,0.0
1980-01-01 03:00:00,1980-01-01T03:00:00Z,-4.076,0.0,0.0,-1.843,0.0,0.0,3.197,0.0,0.0,...,0.0,-4.139,0.0,0.0,-4.202,0.0,0.0,-4.421,0.0,0.0
1980-01-01 04:00:00,1980-01-01T04:00:00Z,-4.248,0.0,0.0,-2.094,0.0,0.0,2.750,0.0,0.0,...,0.0,-4.239,0.0,0.0,-4.732,0.0,0.0,-4.449,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-31 19:00:00,2019-12-31T19:00:00Z,-1.386,0.0,0.0,1.684,0.0,0.0,-1.351,0.0,0.0,...,0.0,-0.382,0.0,0.0,0.053,0.0,0.0,-0.766,0.0,0.0
2019-12-31 20:00:00,2019-12-31T20:00:00Z,-1.661,0.0,0.0,1.718,0.0,0.0,-1.009,0.0,0.0,...,0.0,-0.418,0.0,0.0,-0.207,0.0,0.0,-0.742,0.0,0.0
2019-12-31 21:00:00,2019-12-31T21:00:00Z,-1.986,0.0,0.0,1.726,0.0,0.0,-0.986,0.0,0.0,...,0.0,-0.490,0.0,0.0,-0.203,0.0,0.0,-0.764,0.0,0.0


In [11]:
mask2 = (weather['utc_timestamp'] > "2015-01-01 00:00:00") & (weather['utc_timestamp'] <= "2019-12-31 19:00:00")
weather = weather.loc[mask2]

weather

,utc_timestamp,AT_temperature,AT_radiation_direct_horizontal,AT_radiation_diffuse_horizontal,BE_temperature,BE_radiation_direct_horizontal,BE_radiation_diffuse_horizontal,BG_temperature,BG_radiation_direct_horizontal,BG_radiation_diffuse_horizontal,...,RO_radiation_diffuse_horizontal,SE_temperature,SE_radiation_direct_horizontal,SE_radiation_diffuse_horizontal,SI_temperature,SI_radiation_direct_horizontal,SI_radiation_diffuse_horizontal,SK_temperature,SK_radiation_direct_horizontal,SK_radiation_diffuse_horizontal
utc_timestamp,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00,2015-01-01T00:00:00Z,-6.503,0.0,0.0,-0.046,0.0,0.0,-13.350,0.0,0.0,...,0.0,0.690,0.0,0.0,-11.261,0.0,0.0,-8.272,0.0,0.0
2015-01-01 01:00:00,2015-01-01T01:00:00Z,-6.458,0.0,0.0,-0.302,0.0,0.0,-13.594,0.0,0.0,...,0.0,0.786,0.0,0.0,-10.975,0.0,0.0,-8.114,0.0,0.0
2015-01-01 02:00:00,2015-01-01T02:00:00Z,-6.253,0.0,0.0,-0.541,0.0,0.0,-13.781,0.0,0.0,...,0.0,0.946,0.0,0.0,-10.772,0.0,0.0,-7.898,0.0,0.0
2015-01-01 03:00:00,2015-01-01T03:00:00Z,-5.908,0.0,0.0,-0.794,0.0,0.0,-14.042,0.0,0.0,...,0.0,1.160,0.0,0.0,-10.667,0.0,0.0,-7.670,0.0,0.0
2015-01-01 04:00:00,2015-01-01T04:00:00Z,-5.516,0.0,0.0,-1.053,0.0,0.0,-14.340,0.0,0.0,...,0.0,1.382,0.0,0.0,-10.467,0.0,0.0,-7.399,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-30 19:00:00,2019-12-30T19:00:00Z,-1.966,0.0,0.0,0.381,0.0,0.0,-4.800,0.0,0.0,...,0.0,4.427,0.0,0.0,-2.730,0.0,0.0,-3.966,0.0,0.0
2019-12-30 20:00:00,2019-12-30T20:00:00Z,-1.934,0.0,0.0,0.142,0.0,0.0,-5.148,0.0,0.0,...,0.0,4.245,0.0,0.0,-2.937,0.0,0.0,-4.090,0.0,0.0
2019-12-30 21:00:00,2019-12-30T21:00:00Z,-1.766,0.0,0.0,-0.082,0.0,0.0,-5.467,0.0,0.0,...,0.0,4.055,0.0,0.0,-2.968,0.0,0.0,-4.208,0.0,0.0


In [12]:
weather = weather [['utc_timestamp','AT_temperature', 'AT_radiation_direct_horizontal', 'BE_temperature', 'BE_radiation_direct_horizontal','BG_temperature','BG_radiation_direct_horizontal', 'CH_temperature', 'CH_radiation_direct_horizontal','CZ_temperature', 'CZ_radiation_direct_horizontal','DE_temperature', 'DE_radiation_direct_horizontal', 'DK_temperature', 'DK_radiation_direct_horizontal','ES_temperature', 'ES_radiation_direct_horizontal', 'FR_temperature', 'FR_radiation_direct_horizontal', 'GR_temperature', 'GR_radiation_direct_horizontal','HR_temperature', 'HR_radiation_direct_horizontal', 'HU_temperature', 'HU_radiation_direct_horizontal','IT_temperature', 'IT_radiation_direct_horizontal','LT_temperature','LT_radiation_direct_horizontal', 'NL_temperature', 'NL_radiation_direct_horizontal','PL_temperature', 'PL_radiation_direct_horizontal','PT_temperature', 'PT_radiation_direct_horizontal','RO_temperature', 'RO_radiation_direct_horizontal', 'SI_temperature', 'SI_radiation_direct_horizontal', 'SK_temperature', 'SK_radiation_direct_horizontal']]
weather

,utc_timestamp,AT_temperature,AT_radiation_direct_horizontal,BE_temperature,BE_radiation_direct_horizontal,BG_temperature,BG_radiation_direct_horizontal,CH_temperature,CH_radiation_direct_horizontal,CZ_temperature,...,PL_temperature,PL_radiation_direct_horizontal,PT_temperature,PT_radiation_direct_horizontal,RO_temperature,RO_radiation_direct_horizontal,SI_temperature,SI_radiation_direct_horizontal,SK_temperature,SK_radiation_direct_horizontal
utc_timestamp,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00,2015-01-01T00:00:00Z,-6.503,0.0,-0.046,0.0,-13.350,0.0,-9.152,0.0,-2.372,...,-1.198,0.0,5.584,0.0,-14.323,0.0,-11.261,0.0,-8.272,0.0
2015-01-01 01:00:00,2015-01-01T01:00:00Z,-6.458,0.0,-0.302,0.0,-13.594,0.0,-8.946,0.0,-2.126,...,-1.052,0.0,5.219,0.0,-14.166,0.0,-10.975,0.0,-8.114,0.0
2015-01-01 02:00:00,2015-01-01T02:00:00Z,-6.253,0.0,-0.541,0.0,-13.781,0.0,-8.676,0.0,-1.839,...,-0.925,0.0,5.034,0.0,-14.008,0.0,-10.772,0.0,-7.898,0.0
2015-01-01 03:00:00,2015-01-01T03:00:00Z,-5.908,0.0,-0.794,0.0,-14.042,0.0,-8.373,0.0,-1.521,...,-0.793,0.0,4.708,0.0,-13.963,0.0,-10.667,0.0,-7.670,0.0
2015-01-01 04:00:00,2015-01-01T04:00:00Z,-5.516,0.0,-1.053,0.0,-14.340,0.0,-8.140,0.0,-1.223,...,-0.627,0.0,4.353,0.0,-14.021,0.0,-10.467,0.0,-7.399,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-30 19:00:00,2019-12-30T19:00:00Z,-1.966,0.0,0.381,0.0,-4.800,0.0,-0.464,0.0,-1.713,...,0.078,0.0,10.928,0.0,-4.938,0.0,-2.730,0.0,-3.966,0.0
2019-12-30 20:00:00,2019-12-30T20:00:00Z,-1.934,0.0,0.142,0.0,-5.148,0.0,-0.503,0.0,-1.830,...,-0.141,0.0,10.325,0.0,-5.058,0.0,-2.937,0.0,-4.090,0.0
2019-12-30 21:00:00,2019-12-30T21:00:00Z,-1.766,0.0,-0.082,0.0,-5.467,0.0,-0.361,0.0,-1.785,...,-0.165,0.0,9.728,0.0,-4.995,0.0,-2.968,0.0,-4.208,0.0


In [14]:
df = pd.concat([countries, weather], axis=1)
df

,utc_timestamp,AT_solar_generation_actual,BE_solar_generation_actual,BG_solar_generation_actual,CH_solar_generation_actual,CZ_solar_generation_actual,DE_solar_generation_actual,DK_solar_generation_actual,ES_solar_generation_actual,FR_solar_generation_actual,...,PL_temperature,PL_radiation_direct_horizontal,PT_temperature,PT_radiation_direct_horizontal,RO_temperature,RO_radiation_direct_horizontal,SI_temperature,SI_radiation_direct_horizontal,SK_temperature,SK_radiation_direct_horizontal
utc_timestamp,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.198,0.0,5.584,0.0,-14.323,0.0,-11.261,0.0,-8.272,0.0
2015-01-01 01:00:00,2015-01-01 01:00:00,NaN,NaN,NaN,0.05,NaN,NaN,NaN,50.0,NaN,...,-1.052,0.0,5.219,0.0,-14.166,0.0,-10.975,0.0,-8.114,0.0
2015-01-01 02:00:00,2015-01-01 02:00:00,NaN,NaN,NaN,0.07,NaN,NaN,NaN,50.0,NaN,...,-0.925,0.0,5.034,0.0,-14.008,0.0,-10.772,0.0,-7.898,0.0
2015-01-01 03:00:00,2015-01-01 03:00:00,NaN,NaN,NaN,0.07,NaN,NaN,NaN,42.0,NaN,...,-0.793,0.0,4.708,0.0,-13.963,0.0,-10.667,0.0,-7.670,0.0
2015-01-01 04:00:00,2015-01-01 04:00:00,NaN,NaN,NaN,0.07,NaN,NaN,0.01,34.0,NaN,...,-0.627,0.0,4.353,0.0,-14.021,0.0,-10.467,0.0,-7.399,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-31 15:00:00,2019-12-31 15:00:00,19.0,13.14,0.0,2.40,9.61,46.0,0.73,2999.0,454.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-31 16:00:00,2019-12-31 16:00:00,7.0,0.00,0.0,0.00,0.00,0.0,0.00,1250.0,109.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-31 17:00:00,2019-12-31 17:00:00,4.0,0.00,0.0,0.00,0.00,0.0,0.00,182.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df['timestamp'] = df.index
df

,utc_timestamp,AT_solar_generation_actual,BE_solar_generation_actual,BG_solar_generation_actual,CH_solar_generation_actual,CZ_solar_generation_actual,DE_solar_generation_actual,DK_solar_generation_actual,ES_solar_generation_actual,FR_solar_generation_actual,...,PL_radiation_direct_horizontal,PT_temperature,PT_radiation_direct_horizontal,RO_temperature,RO_radiation_direct_horizontal,SI_temperature,SI_radiation_direct_horizontal,SK_temperature,SK_radiation_direct_horizontal,timestamp
utc_timestamp,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,5.584,0.0,-14.323,0.0,-11.261,0.0,-8.272,0.0,2015-01-01 00:00:00
2015-01-01 01:00:00,2015-01-01 01:00:00,NaN,NaN,NaN,0.05,NaN,NaN,NaN,50.0,NaN,...,0.0,5.219,0.0,-14.166,0.0,-10.975,0.0,-8.114,0.0,2015-01-01 01:00:00
2015-01-01 02:00:00,2015-01-01 02:00:00,NaN,NaN,NaN,0.07,NaN,NaN,NaN,50.0,NaN,...,0.0,5.034,0.0,-14.008,0.0,-10.772,0.0,-7.898,0.0,2015-01-01 02:00:00
2015-01-01 03:00:00,2015-01-01 03:00:00,NaN,NaN,NaN,0.07,NaN,NaN,NaN,42.0,NaN,...,0.0,4.708,0.0,-13.963,0.0,-10.667,0.0,-7.670,0.0,2015-01-01 03:00:00
2015-01-01 04:00:00,2015-01-01 04:00:00,NaN,NaN,NaN,0.07,NaN,NaN,0.01,34.0,NaN,...,0.0,4.353,0.0,-14.021,0.0,-10.467,0.0,-7.399,0.0,2015-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-31 15:00:00,2019-12-31 15:00:00,19.0,13.14,0.0,2.40,9.61,46.0,0.73,2999.0,454.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-12-31 15:00:00
2019-12-31 16:00:00,2019-12-31 16:00:00,7.0,0.00,0.0,0.00,0.00,0.0,0.00,1250.0,109.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-12-31 16:00:00
2019-12-31 17:00:00,2019-12-31 17:00:00,4.0,0.00,0.0,0.00,0.00,0.0,0.00,182.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-12-31 17:00:00


In [16]:
dcolumns = [column for column in df.columns if ('utc_timestamp' in column or 'HR' in column or 'HU' in column or 'PL' in column)]
dataset = df.drop(columns=dcolumns)
dataset

,AT_solar_generation_actual,BE_solar_generation_actual,BG_solar_generation_actual,CH_solar_generation_actual,CZ_solar_generation_actual,DE_solar_generation_actual,DK_solar_generation_actual,ES_solar_generation_actual,FR_solar_generation_actual,GR_solar_generation_actual,...,NL_radiation_direct_horizontal,PT_temperature,PT_radiation_direct_horizontal,RO_temperature,RO_radiation_direct_horizontal,SI_temperature,SI_radiation_direct_horizontal,SK_temperature,SK_radiation_direct_horizontal,timestamp
utc_timestamp,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,5.584,0.0,-14.323,0.0,-11.261,0.0,-8.272,0.0,2015-01-01 00:00:00
2015-01-01 01:00:00,NaN,NaN,NaN,0.05,NaN,NaN,NaN,50.0,NaN,NaN,...,0.0,5.219,0.0,-14.166,0.0,-10.975,0.0,-8.114,0.0,2015-01-01 01:00:00
2015-01-01 02:00:00,NaN,NaN,NaN,0.07,NaN,NaN,NaN,50.0,NaN,NaN,...,0.0,5.034,0.0,-14.008,0.0,-10.772,0.0,-7.898,0.0,2015-01-01 02:00:00
2015-01-01 03:00:00,NaN,NaN,NaN,0.07,NaN,NaN,NaN,42.0,NaN,NaN,...,0.0,4.708,0.0,-13.963,0.0,-10.667,0.0,-7.670,0.0,2015-01-01 03:00:00
2015-01-01 04:00:00,NaN,NaN,NaN,0.07,NaN,NaN,0.01,34.0,NaN,NaN,...,0.0,4.353,0.0,-14.021,0.0,-10.467,0.0,-7.399,0.0,2015-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-31 15:00:00,19.0,13.14,0.0,2.40,9.61,46.0,0.73,2999.0,454.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-12-31 15:00:00
2019-12-31 16:00:00,7.0,0.00,0.0,0.00,0.00,0.0,0.00,1250.0,109.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-12-31 16:00:00
2019-12-31 17:00:00,4.0,0.00,0.0,0.00,0.00,0.0,0.00,182.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-12-31 17:00:00


In [20]:
#list of dataframes - one for each country
#8 more countries added
dfs1 = [] 

countries = ['BE', 'DK', 'FR', 'IT', 'LT', 'NL', 'PT', 'SK', 'GR', 'CH', 'AT', 'BG', 'CZ', 'DE', 'ES', 'RO']

for country in countries:
    columns = [column for column in dataset.columns if ('timestamp' in column or country in column or 'month' in column or 'month_sin' in column or 'month_cos' in column) ]
    dfs1.append(dataset[columns])

print(f'{len(dfs1)} countries')
print(dfs1[0])

16 countries
                     BE_solar_generation_actual  month  month_sin  month_cos  \
utc_timestamp                                                                  
2015-01-01 00:00:00                         NaN    NaN        NaN        NaN   
2015-01-01 01:00:00                         NaN    1.0   0.000249   1.000000   
2015-01-01 02:00:00                         NaN    1.0   0.000374   1.000000   
2015-01-01 03:00:00                         NaN    1.0   0.000499   1.000000   
2015-01-01 04:00:00                         NaN    1.0   0.000623   1.000000   
...                                         ...    ...        ...        ...   
2019-12-31 15:00:00                       13.14   12.0  -0.731768   0.681554   
2019-12-31 16:00:00                        0.00   12.0  -0.731683   0.681645   
2019-12-31 17:00:00                        0.00   12.0  -0.731598   0.681737   
2019-12-31 18:00:00                        0.00   12.0  -0.731513   0.681828   
2019-12-31 19:00:00        

In [21]:
dfs1[13]

,DE_solar_generation_actual,month,month_sin,month_cos,DE_temperature,DE_radiation_direct_horizontal,timestamp
utc_timestamp,,,,,,,
2015-01-01 00:00:00,NaN,NaN,NaN,NaN,-0.981,0.0,2015-01-01 00:00:00
2015-01-01 01:00:00,NaN,1.0,0.000249,1.000000,-1.035,0.0,2015-01-01 01:00:00
2015-01-01 02:00:00,NaN,1.0,0.000374,1.000000,-1.109,0.0,2015-01-01 02:00:00
2015-01-01 03:00:00,NaN,1.0,0.000499,1.000000,-1.166,0.0,2015-01-01 03:00:00
2015-01-01 04:00:00,NaN,1.0,0.000623,1.000000,-1.226,0.0,2015-01-01 04:00:00
...,...,...,...,...,...,...,...
2019-12-31 15:00:00,46.0,12.0,-0.731768,0.681554,NaN,NaN,2019-12-31 15:00:00
2019-12-31 16:00:00,0.0,12.0,-0.731683,0.681645,NaN,NaN,2019-12-31 16:00:00
2019-12-31 17:00:00,0.0,12.0,-0.731598,0.681737,NaN,NaN,2019-12-31 17:00:00


In [22]:
BE_dataset = dfs1[0]
DK_dataset = dfs1[1]
FR_dataset = dfs1[2]
IT_dataset = dfs1[3]
LT_dataset = dfs1[4]
NL_dataset = dfs1[5]
PT_dataset = dfs1[6]
SK_dataset = dfs1[7]
GR_dataset = dfs1[8]
SW_dataset = dfs1[9]
AU_dataset = dfs1[10]
BG_dataset = dfs1[11]
CZ_dataset = dfs1[12]
DE_dataset = dfs1[13]
ES_dataset = dfs1[14]
RO_dataset = dfs1[15]

In [23]:
BE_dataset.fillna(method='bfill', inplace=True)
DK_dataset.fillna(method='bfill', inplace=True)
FR_dataset.fillna(method='bfill', inplace=True)
IT_dataset.fillna(method='bfill', inplace=True)
LT_dataset.fillna(method='bfill', inplace=True)
NL_dataset.fillna(method='bfill', inplace=True)
PT_dataset.fillna(method='bfill', inplace=True)
SK_dataset.fillna(method='bfill', inplace=True)
GR_dataset.fillna(method='bfill', inplace=True)
SW_dataset.fillna(method='bfill', inplace=True)
AU_dataset.fillna(method='bfill', inplace=True)
BG_dataset.fillna(method='bfill', inplace=True)
CZ_dataset.fillna(method='bfill', inplace=True)
DE_dataset.fillna(method='bfill', inplace=True)
ES_dataset.fillna(method='bfill', inplace=True)
RO_dataset.fillna(method='bfill', inplace=True)

In [26]:
AU_dataset

,AT_solar_generation_actual,month,month_sin,month_cos,AT_temperature,AT_radiation_direct_horizontal,timestamp
utc_timestamp,,,,,,,
2015-01-01 00:00:00,2.0,1.0,0.000249,1.000000,-6.503,0.0,2015-01-01 00:00:00
2015-01-01 01:00:00,2.0,1.0,0.000249,1.000000,-6.458,0.0,2015-01-01 01:00:00
2015-01-01 02:00:00,2.0,1.0,0.000374,1.000000,-6.253,0.0,2015-01-01 02:00:00
2015-01-01 03:00:00,2.0,1.0,0.000499,1.000000,-5.908,0.0,2015-01-01 03:00:00
2015-01-01 04:00:00,2.0,1.0,0.000623,1.000000,-5.516,0.0,2015-01-01 04:00:00
...,...,...,...,...,...,...,...
2019-12-31 15:00:00,19.0,12.0,-0.731768,0.681554,NaN,NaN,2019-12-31 15:00:00
2019-12-31 16:00:00,7.0,12.0,-0.731683,0.681645,NaN,NaN,2019-12-31 16:00:00
2019-12-31 17:00:00,4.0,12.0,-0.731598,0.681737,NaN,NaN,2019-12-31 17:00:00


In [26]:
BE_series = darts.TimeSeries.from_series(BE_dataset.filter(['BE_solar_generation_actual'], axis=1))
BE_past = darts.TimeSeries.from_dataframe(BE_dataset.drop(['BE_solar_generation_actual', 'timestamp'], axis=1))

DK_series = darts.TimeSeries.from_series(DK_dataset.filter(['DK_solar_generation_actual'], axis=1))
DK_past = darts.TimeSeries.from_dataframe(DK_dataset.drop(['DK_solar_generation_actual', 'timestamp'], axis=1))

FR_series = darts.TimeSeries.from_series(FR_dataset.filter(['FR_solar_generation_actual'], axis=1))
FR_past = darts.TimeSeries.from_dataframe(FR_dataset.drop(['FR_solar_generation_actual', 'timestamp'], axis=1))

IT_series = darts.TimeSeries.from_series(IT_dataset.filter(['IT_solar_generation_actual'], axis=1))
IT_past = darts.TimeSeries.from_dataframe(IT_dataset.drop(['IT_solar_generation_actual', 'timestamp'], axis=1))

LT_series = darts.TimeSeries.from_series(LT_dataset.filter(['LT_solar_generation_actual'], axis=1))
LT_past = darts.TimeSeries.from_dataframe(LT_dataset.drop(['LT_solar_generation_actual', 'timestamp'], axis=1))

NL_series = darts.TimeSeries.from_series(NL_dataset.filter(['NL_solar_generation_actual'], axis=1))
NL_past = darts.TimeSeries.from_dataframe(NL_dataset.drop(['NL_solar_generation_actual', 'timestamp'], axis=1))

PT_series = darts.TimeSeries.from_series(PT_dataset.filter(['PT_solar_generation_actual'], axis=1))
PT_past = darts.TimeSeries.from_dataframe(PT_dataset.drop(['PT_solar_generation_actual', 'timestamp'], axis=1))

SK_series = darts.TimeSeries.from_series(SK_dataset.filter(['SK_solar_generation_actual'], axis=1))
SK_past = darts.TimeSeries.from_dataframe(SK_dataset.drop(['SK_solar_generation_actual', 'timestamp'], axis=1))

GR_series = darts.TimeSeries.from_series(GR_dataset.filter(['GR_solar_generation_actual'], axis=1))
GR_past = darts.TimeSeries.from_dataframe(GR_dataset.drop(['GR_solar_generation_actual', 'timestamp'], axis=1))

SW_series = darts.TimeSeries.from_series(SW_dataset.filter(['CH_solar_generation_actual'], axis=1))
SW_past = darts.TimeSeries.from_dataframe(SW_dataset.drop(['CH_solar_generation_actual', 'timestamp'], axis=1))

AU_series = darts.TimeSeries.from_series(AU_dataset.filter(['AT_solar_generation_actual'], axis=1))
AU_past = darts.TimeSeries.from_dataframe(AU_dataset.drop(['AT_solar_generation_actual', 'timestamp'], axis=1))

BG_series = darts.TimeSeries.from_series(BG_dataset.filter(['BG_solar_generation_actual'], axis=1))
BG_past = darts.TimeSeries.from_dataframe(BG_dataset.drop(['BG_solar_generation_actual', 'timestamp'], axis=1))

CZ_series = darts.TimeSeries.from_series(CZ_dataset.filter(['CZ_solar_generation_actual'], axis=1))
CZ_past = darts.TimeSeries.from_dataframe(CZ_dataset.drop(['CZ_solar_generation_actual', 'timestamp'], axis=1))

DE_series = darts.TimeSeries.from_series(DE_dataset.filter(['DE_solar_generation_actual'], axis=1))
DE_past = darts.TimeSeries.from_dataframe(DE_dataset.drop(['DE_solar_generation_actual', 'timestamp'], axis=1))

ES_series = darts.TimeSeries.from_series(ES_dataset.filter(['ES_solar_generation_actual'], axis=1))
ES_past = darts.TimeSeries.from_dataframe(ES_dataset.drop(['ES_solar_generation_actual', 'timestamp'], axis=1))

RO_series = darts.TimeSeries.from_series(RO_dataset.filter(['RO_solar_generation_actual'], axis=1))
RO_past = darts.TimeSeries.from_dataframe(RO_dataset.drop(['RO_solar_generation_actual', 'timestamp'], axis=1))

In [27]:
series_train = [
    BE_series[:-365*24],
    DK_series[:-365*24],
    FR_series[:-365*24],
    IT_series[:-365*24],
    LT_series[:-365*24],
    NL_series[:-365*24],
    PT_series[:-365*24],
    SK_series[:-365*24],
]

series_train_past = [
    BE_past[:-365*24],
    DK_past[:-365*24],
    FR_past[:-365*24],
    IT_past[:-365*24],
    LT_past[:-365*24],
    NL_past[:-365*24],
    PT_past[:-365*24],
    SK_past[:-365*24],
]
            
series_test = {
    'BE': BE_series[-365*24:],
    'DK': DK_series[-365*24:],
    'FR': FR_series[-365*24:],
    'IT': IT_series[-365*24:],
    'LT': LT_series[-365*24:],
    'NL': NL_series[-365*24:],
    'PT': PT_series[-365*24:],
    'SK': SK_series[-365*24:],
    'AU': AU_series[-365*24:],
    'BG': BG_series[-365*24:],
    'SW': SW_series[-365*24:],
    'CZ': CZ_series[-365*24:],
    'DE': DE_series[-365*24:],
    'ES': ES_series[-365*24:],
    'GR': GR_series[-365*24:],
    'RO': RO_series[-365*24:],
}

series_test_past = [
    BE_past[-365*24:],
    DK_past[-365*24:],
    FR_past[-365*24:],
    IT_past[-365*24:],
    LT_past[-365*24:],
    NL_past[-365*24:],
    PT_past[-365*24:],
    SK_past[-365*24:],
    AU_past[-365*24:],
    BG_past[-365*24:],
    SW_past[-365*24:],
    CZ_past[-365*24:],
    DE_past[-365*24:],
    ES_past[-365*24:],
    GR_past[-365*24:],
    RO_past[-365*24:],
]

In [28]:
torch.cuda.is_available()

True

In [29]:
torch.set_float32_matmul_precision('medium')
model = NBEATSModel(input_chunk_length=24, output_chunk_length=24, pl_trainer_kwargs={"accelerator": "gpu", 'devices': [0]}, batch_size=2048)
model.fit(series=series_train, past_covariates=series_train_past, epochs=100, verbose=True)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | stacks        | ModuleList       | 7.2 M 
---------------------------------------------------
7.2 M     Trainable params
2.1 K     Non-trainable params
7.2 M     Total params
28.655    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


In [30]:
import logging
logging.getLogger("pytorch_lightning").setLevel(logging.WARNING)
results = {}
for i, country in enumerate(series_test):
    backtest1 = model.backtest(series=series_test[country], past_covariates=series_test_past[i], verbose=True, forecast_horizon=24, stride=24, retrain=False, last_points_only=False, metric=[darts.metrics.metrics.rmse])
    results[country] = backtest1

  0%|          | 0/364 [00:00<?, ?it/s]

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

...

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

In [31]:
results

{'BE': 292.7299724759668,
 'DK': 92.48536533733912,
 'FR': 432.23972994810583,
 'IT': 775.95880723915,
 'LT': 9.114168767912888,
 'NL': 42.09295670666652,
 'PT': 48.24619446415142,
 'SK': 43.06932662823052,
 'AU': 133.36537227333108,
 'BG': 93.14540470248865,
 'SW': 50.25873973757772,
 'CZ': 227.67613567199936,
 'DE': 1927.6173165402809,
 'ES': 537.5437892562713,
 'GR': 185.01207600180575,
 'RO': 93.12808138919176}